# TP1 
## Estruturas Criptográficas - Criptografia e Segurança da Informação

### Exercício 2

2. Use o “package” Cryptography para
    1. Implementar uma AEAD com “Tweakable Block Ciphers” conforme está descrito na última secção do texto [+Capítulo 1: Primitivas Criptográficas Básicas](https://paper.dropbox.com/doc/Capitulo-1-Primitivas-Criptograficas-Basicas-YAcE9VWuF88R2fmPyvKlx#:uid=971079522289346670472132&h2=AEAD-com-%E2%80%9CTweakable-Block-Ciph).  A cifra por blocos primitiva, usada para gerar a “tweakable block cipher”, é o AES-256 ou o ChaCha20.
    2. Use esta cifra para construir um canal privado de informação assíncrona com acordo de chaves feito com “X448 key exchange” e “Ed448 Signing&Verification” para autenticação  dos agentes. Deve incluir uma fase de confirmação da chave acordada.


##### Instalar packages necessários

In [377]:
#%pip install cryptography

##### Resumo 

Este 1º trabalho prático da Unidade Curricular de Estruturas Criptográficas tem como objetivo a implementação de uma AEAD com "Tweakable Block Ciphers" e a construção de um canal privado de informação assíncrona com acordo de chaves feito com "X448 key exchange" e "Ed448 Signing&Verification" para autenticação dos agentes, incluindo uma fase de confirmação da chave acordada.

A ordem deste notebook será a seguinte:

Numa primeira secção é demonstrado o código desenvolvido bem como a sua explicação, divididos em secções. De seguida é implementada uma série de testes ao que foi desenvolvido.


#### Imports

Sobre os imports é necessário mencionar o seguinte : 

- Tal como imposto no enunciado, foi utilizado o package "cryptography" para a implementação de todas as funcionalidades pedidas.

- Utilização do asyncio para simular a comunicação entre duas entidades.

- Biblioteca OS e utilização do [os.urandom](https://cryptography.io/en/stable/random-numbers/) para gerar valores aleatórios, uma vez que é uma melhora forma de geral números aleatórios do que a função random do python.

- Import do datetime para a obtenção do tempo atual, para a geração da `associated data`

In [378]:
import os
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric.x448 import X448PrivateKey
from cryptography.hazmat.primitives.asymmetric.ed448 import Ed448PrivateKey
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives.serialization import Encoding, PublicFormat
from cryptography.hazmat.primitives.ciphers.aead import ChaCha20Poly1305
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from datetime import datetime
import sys
import asyncio
import nest_asyncio

nest_asyncio.apply()

#### Modo [TAE (“tweaked authentication encryption”)](https://shorturl.at/fpsB7)

Para a resolução deste trabalho, optamos por duas abordagens diferentes, uma para o AES e outra para o ChaCha20. Estas abordagens têm também diferença se implementam o modo TAE ou não (assim como descrito na última secção do texto [Capítulo 1: Primitivas Criptográficas Básicas](https://paper.dropbox.com/doc/Capitulo-1-Primitivas-Criptograficas-Basicas-YAcE9VWuF88R2fmPyvKlx#:uid=971079522289346670472132&h2=AEAD-com-%E2%80%9CTweakable-Block-Ciph)).

Optamos por utilizar o Chacha20 com o modo TAE e o AES sem o modo TAE.

Esta opção é ativada através de um parâmetro boleano que é passado à classe `channel`. Se obter o valor de `True` então o modo TAE está ativado e é utilizado o __Chacha20Poly1305__, caso contrário é utilizado o __AES256__.

DIFERENÇA ENTRE TAE E OUTROS ADJACENTES !!!! ESCREVER O DEBAIXO NOUTRAS PALAVRAS

A cifra AEAD com “Tweakable Block Ciphers” utiliza um input adicional designado de *tweak*. Estes funcionam como chaves únicas de cada bloco, enquanto que a chave propriamente dita é a mesma em todos os blocos, tornando a cifra menos vulnerável a ataques.


In [379]:
mode_TAE = True

#### Funções auxiliares sem modo TAE

VALORES OBTIDOS EMAIL PROENÇA ! + chave de longa e de curta duração

Estas funções servem de auxiliares quando o __modo TAE__ toma o valor de falso. Neste caso e de modo a apoiar a expressão $ Ẽ(w,k,x) = E(k,w ⊕ E(k,x)) $, é utilizado uma função de padding. Esta função primeiramente aumenta o tamanho do tweak para o tamanho da mensagem, preenchendo o tweak de bits de valor 0 e de seguida chama a função `xor` para fazer ⊕ entre cada bit da primeira cifra e do tweak.

In [380]:
def padding(b1,b2):
    lb1 = len(b1)
    lb2 = len(b2)
    if lb1 < lb2:
        b1 += b"\x00" * (lb2 - lb1)
    return xor(b1, b2)

def xor(b1, b2): 
    result = b''
    result += bytes([bt1 ^ bt2 for bt1, bt2 in zip(b1,b2) ]) # for b1, b2 in zip(b1, b2):
    return result

#### Funções auxiliares com modo TAE

De modo a aplicar o modo TAE, é preciso primeiro uma função auxiliar que gera os tweaks a aplicar, assim como a [figura demonstrada nas aulas](https://paper-attachments.dropbox.com/s_B1054BDA5AC312A4FB9A5763C788D22B1CCD183EF0DF33A2A6FDC7FDD5358446_1615655577238_Tela+2.png). 


Esta função recebe como input  o número de blocos, tamanho da mensagem, e um “name only used once” (nonce). Esta função vai gerar os vários tweaks gerados por esta função serão utilizados no mecanismo geral da cifra que vai ser explicado mais à frente (função encrypt e decrypt). O n-1 primeiros blocos, onde n é o número total de blocos, são utilizados para a cifra dos blocos da mensagem e último é utilizado para a autenticação do criptograma.

Decidimos que cada bloco vai ter 32 bytes, sendo igual ao tamanhos dos blocos das mensagens.

Tal como descrito na imagem acima mencionada nos n-1 primeiros blocos são gerados da seguinte maneira:

- A primeira metade é ocupada pelo nounce (16 bytes).
- A segunda metade é ocupada pelo contador (número do bloco que está a ser cifrado)
- Por último, um bit a 0.

$w_i = [nonce|i|0]$, i = 0.. n-1

Já o último bloco é gerado de forma diferente:

- A primeira metade é ocupada pelo nounce (16 bytes).
- O comprimento da mensagem (sem padding)
- Um bit a 1

$w^{*} = [nonce | length(plaintext) | 1]$



In [381]:
def gen_tweaks(number_of_blocks, plaintext_length, nonce):
    cipher_tweaks = []
    # cipher tweaks [nonce|counter|0]
    for i in range(0, number_of_blocks):
        tweak = nonce + int(i).to_bytes(16, byteorder='big')
        tweak = int.from_bytes(tweak, byteorder='big')

        # remove last bit and add the final bit 0
        tweak = tweak >> 1
        tweak = tweak << 1

        tweak = tweak.to_bytes(32, byteorder='big')
        cipher_tweaks.append(tweak)
    
    # authentication tweak [nonce|plaintext_length|0]
    auth_tweak = nonce + plaintext_length.to_bytes(16, byteorder='big')
    auth_tweak = int.from_bytes(auth_tweak, byteorder='big')

    # last bit of auth_tweak to 1
    mask = 0b1
    auth_tweak = auth_tweak | mask
    auth_tweak = auth_tweak.to_bytes(32, byteorder='big')

    return cipher_tweaks, auth_tweak


#### 

Esta função é utilizada para cifrar cada bloco da mensagem. A mensagem é cifrada de acordo com a abordagem acima descrita e ensinada na aula: $ Ẽ(w,k,x) = E(k,w ⊕ E(k,x)) $.

Esta função devolve Ẽ(w,k,x). Utilizamos o algoritmo `ChaCha20Poly1305` para cifrar.

# ... COMPLETAR ISTO 


In [382]:
def get_ciphertext(cipher_key, nonce, tweak, plaintext, ad):

    chacha = ChaCha20Poly1305(cipher_key)
    ciphertext = chacha.encrypt(nonce, plaintext, ad)

    xored = b''

    for (a,b) in zip(tweak, ciphertext):
        xored += bytes([a^b])

    return xored

Esta função funciona de maneira análoga à função de cifrar descrita posteriormente.

Recebe-se como argumentos para a função o criptograma, tag, o nounce utilizado para gerar o tweak, a chave e os dados associados.

Primeiramente, dividimos o criptograma em blocos de 32 bytes.
Vamos depois com o número de blocos, o tamanho da mensagem (plaintext) e o nounce utilizado para gerar os tweaks. 

De seguida vamos decifrar ca 

In [383]:

def decrypt(ciphertext, tag, nonce, nonce_tweak, cipher_key, ad):
    # divide plaintext into blocks
    blocks = [ciphertext[i:i+32] for i in range(0, len(ciphertext), 32)]

    number_of_blocks = len(blocks)
    n = len(ciphertext)
    r = len(tag)
    length = n - (32 - r)

    # generate tweaks
    cipher_tweaks, auth_tweak = gen_tweaks(number_of_blocks, length, nonce_tweak)

    decrypted_blocks = []

    # decrypt blocks
    for w in range(0, number_of_blocks):
        plaintext = get_ciphertext(cipher_key, nonce, cipher_tweaks[w], blocks[w], ad)
        decrypted_blocks.append(plaintext)   
    
    # authentication phase
    auth = decrypted_blocks[0]
    for i in range(1, number_of_blocks):
        xored = [(a^b).to_bytes(1,byteorder='big') for (a,b) in zip(auth, decrypted_blocks[i])]
        auth = b"".join(xored)

    auth = auth[:r]
    auth_tweak = auth_tweak[:r]

    generated_tag = get_ciphertext(cipher_key, nonce, auth_tweak, auth, ad)

    # verify authentication
    if tag == generated_tag:
        decrypted_blocks[number_of_blocks - 1] = decrypted_blocks[number_of_blocks - 1][:r]
        plaintext = b"".join(decrypted_blocks)

    else :
        return "ERROR! Different tag used in authentication."
        
    return plaintext.decode('utf-8')


Por último, vem a função de cifrar a mensagem. Primeiramente, devidimos a mensagem em blocos de 32 bytes. O último bloco, caso tenha tamanho inferior a 32, é feito padding coom bits a 0.

Depois de feito a divisão passamos para a geração dos tweaks. Para isso precisamos antes de gerar um nonce a ser utilizado para a geração dos tweaks anteriormente explicado. 

Geramos também o nounce a utilizar com o `ChaCha20Poly1305`. Para cada um dos blocos da mensagem ciframos e juntamos a uma lista de blocos cifrados. Fazemos isto para os n-1 primeiro blocos. 

Para o último bloco, passamos o tamanho do bloco para uma lista de bytes de tamanho 32, ciframos e fazemos o xor com o último bloco da mensagem (plaintext) com pad. 

A última fase que temos de fazer é a de gerar o tag. Para isso primeiro temos que gerar o auth. Este é auth é gerado através do xor de cada um dos blocos da mensagem (plaintext) com o seguinte.  Pegamos nisso e no auth_tweak gerado anterior mente e fazemos a cifragem. Ao ir buscar a tag ignoramos os bytes que foram adicionados com o padding. 

Este passos todos estão como demonstrados na figura [presente na página da Unidade Curricular](https://paper-attachments.dropbox.com/s_B1054BDA5AC312A4FB9A5763C788D22B1CCD183EF0DF33A2A6FDC7FDD5358446_1615655562499_Canvas+1.png).



In [384]:
def encrypt(plaintext, cipher_key, ad):
    # divide plaintext into blocks
    blocks = []
    for i in range(0, len(plaintext), 32):
        block = plaintext[i:i+32].encode('utf-8')
        # padding
        r = len(block) # no final vai dar o comprimento do último bloco
        if r < 32:
            blocks.append(block + ((32 - r) * b'\0'))
        else:
            blocks.append(block)
    
    length = len(plaintext)
    number_of_blocks = len(blocks)

    # generate tweaks
    nonce_tweak = os.urandom(16)
    cipher_tweaks, auth_tweak = gen_tweaks(number_of_blocks, length, nonce_tweak)

    encrypted_blocks = []

    nonce = os.urandom(12)
    
    # encrypt first m-1 blocks
    for w in range(0, number_of_blocks - 1):
        ciphertext = get_ciphertext(cipher_key, nonce, cipher_tweaks[w], blocks[w], ad)
        encrypted_blocks.append(ciphertext)
        
    # encrypt last block
    r_in_bytes = int(r).to_bytes(32, byteorder='big')
    ct = get_ciphertext(cipher_key, nonce, cipher_tweaks[number_of_blocks-1], r_in_bytes, ad)

    xored = [(a^b).to_bytes(1,byteorder='big') for (a,b) in zip(ct, blocks[number_of_blocks-1])]
    last_ciphertext = b"".join(xored)
    
    encrypted_blocks.append(last_ciphertext)

    # authentication phase
    auth = blocks[0]
    for i in range(1, number_of_blocks):
        xored = [(a^b).to_bytes(1,byteorder='big') for (a,b) in zip(auth, blocks[i])]
        auth = b"".join(xored)


    tag = get_ciphertext(cipher_key, nonce, auth_tweak, auth, ad)[:r]

    # join all encrypted blocks
    ciphertext = b"".join(encrypted_blocks)

    return ciphertext, tag, nonce, nonce_tweak

#### Classe Channel

De modo a responder ao segundo ponto pedido na resolução deste exercicio, foi criada uma classe __channel__ ... MUDAR NOMES DITO, ETC !!!

É importante dizer que esta classe é utilizada por ambos os modos, tendo comportamentos semelhantes independentemente do valor da flag __mode TAE__. 

Vamos explicar cada uma destes métodos de modo mais teórico e na secção da função `main`, será explicada a chamada de cada um dos métodos. Deste modo a classe tem 6 métodos:

- O primeiro método é utilizado para criar uma instância do objeto da classe e recebe como argumento um objeto queue, que utilizamos na resolução deste exercício. Esta queue é utilizada uma vez que decidimos para a resolução deste exercício `asyncio`. Mais à frente, a quando da inicialização desta classe.

- Por segundo está o método de gerar as chaves. Neste método e tal como o nome diz, geramos as chaves tanto da primitiva X448 como da primitiva Ed448. 

- Depois de geradas as chaves, criamos um método para partilhar as chaves. Este método coloca na queue do asyncio as chaves públicas (apenas esta podem ser partilhadas). É de realçar também que depois de cada chave pomos também uma assinatura ... __EXPLICAR ISTO MELHOR VER CENA__

- O terceiro método é o de receber as chaves. Este método começa primeiro por receber as chaves públicas e as suas assinaturas. Utiliza a função `verify` para verificar se ou a assinatura ou chave foi alterada durante a sua tranmissão. Se foi, é lançada uma `Exception` que termina o programa. Feito isto, deriva-se um segredo através da utilização da nossa chave privada e da pública do peer (utilizador com o qual estamos a comunicar). Este segredo, e como apenas são comunicadas as chaves públicas, é apenas conhecido pelas partes que vão comunicar. No entanto, este segredo é uma sequência longa de bits. Para combater isto e para combinar a chave a utilizar no AEAD, foi utilizada um `HKDF` para derivar uma chave a utilizar. Este algoritmo usa uma função de hash também para tornar a chave derivada mais dificil de "advinhar" por um atacante.  (__VER EXPLICAÇÃO CHATGPT__)

- Nestes 3 métodos, criamos a base para esta e para o seguinte, o de enviar e receber mensagens respetivamente. A função começa pela criação de `associated data` que é a codificação em bits da hora e dia em que vai ser mandada a mensagem. De seguida e dependendo se o modo TAE está ativo ou não são mandadas mais ou menos dados. No entanto, o desenho geral é o mesmo: Primeiro uma atribuição da key para a chave combinada na função anterior; De seguida, cifragem da mensagem; Por último, colocação da mensagem e diversas informações na queue do `asyncio`.

- O método de receção da mensagem tem uma lógica semelhante ao método de envio. Há uma primeira fase de receção das várias informações da queue do `asyncio`; decifrar o criptograma de acordo com o modo TAE; Por último imprime o a mensagem decifrada.  

- Em último lugar está uma função simples feita para efeitos de debug e para testes, em que imprime a chave concordada no 3º método.

É de mencionar que utilizamos para um modo TAE a verdade utilizamos a primitiva `ChaCha20Poly1305`. Por outro lado, o modo TAE falso utiliza o `AES256` ...

__Decidimos em baixo comentar melhor cada fase dos vários métodos para melhor compreensão.__

In [385]:
class channel:
    def __init__(self, queue):
        self.queue = queue

    async def gen_keys(self):
        
        self.priv_Ed448_key = Ed448PrivateKey.generate()
        self.pub_Ed448_key = self.priv_Ed448_key.public_key()
        
        self.priv_x448_key = X448PrivateKey.generate()
        self.pub_x448_key = self.priv_x448_key.public_key()


    async def share_keys(self):
        
        await self.queue.put(self.pub_Ed448_key)
        
        sigEd448 = self.priv_Ed448_key.sign(
            self.pub_Ed448_key.public_bytes(Encoding.Raw, PublicFormat.Raw)
        )
        
        await self.queue.put(sigEd448)

        
        await self.queue.put(self.pub_x448_key)
        
        sigx448 = self.priv_Ed448_key.sign(
            self.pub_x448_key.public_bytes(Encoding.Raw, PublicFormat.Raw)
        )
        
        await self.queue.put(sigx448)



    async def receive_keys(self):
        
        peer_pub_Ed448_key = await self.queue.get()
        peer_pub_Ed448_key_signature = await self.queue.get()
        
        
        peer_pub_Ed448_key.verify(peer_pub_Ed448_key_signature, peer_pub_Ed448_key.public_bytes(Encoding.Raw, PublicFormat.Raw))
        self.peer_verify_key = peer_pub_Ed448_key
        
        
        peer_pub_x448_key = await self.queue.get()
        peer_pub_x448_key_signature = await self.queue.get()
        
        self.peer_verify_key.verify(peer_pub_x448_key_signature, peer_pub_x448_key.public_bytes(Encoding.Raw, PublicFormat.Raw))
        
        
        shared_secret = self.priv_x448_key.exchange(peer_pub_x448_key)
        derived_key = HKDF(
            algorithm = hashes.SHA256(),
            length = 32, 
            salt = None,
            info = b"handshake data",
        ).derive(shared_secret)
        self.agreed_key = derived_key

    
    async def send(self, plaintext):
        ad = str(datetime.now()).encode('utf-8')

        key = self.agreed_key

        
                
        print("Plaintext Sent: "+str(plaintext))

        if mode_TAE:
            ciphertext,tag,nounce,tweak = encrypt(plaintext, key, ad)
        
        else:
            plaintext = plaintext.encode()
            nounce = os.urandom(16)
            tweak = os.urandom(8)
            aes = Cipher(algorithms.AES256(key), modes.CTR(nounce)).encryptor() 
            ciphertext = aes.update(plaintext)
            xored = padding(tweak, ciphertext)
            ciphertext = aes.update(xored) + aes.finalize()

        
        print("\tCiphertext Sent: "+str(ciphertext))
        
        if mode_TAE: 

            await self.queue.put(self.priv_Ed448_key.sign(tag))
            await self.queue.put(tag)
            
            await self.queue.put(self.priv_Ed448_key.sign(ad))
            await self.queue.put(ad)

        await self.queue.put(self.priv_Ed448_key.sign(ciphertext))
        await self.queue.put(ciphertext)
        

        await self.queue.put(self.priv_Ed448_key.sign(nounce))
        await self.queue.put(nounce)

        await self.queue.put(self.priv_Ed448_key.sign(tweak))
        await self.queue.put(tweak)
        


    async def receive(self):



        key = self.agreed_key

        if mode_TAE : 


            sig_tag = await self.queue.get()
            tag = await self.queue.get()
            self.peer_verify_key.verify(sig_tag, tag)

            adsig = await self.queue.get()
            ad = await self.queue.get()
            self.peer_verify_key.verify(adsig, ad)
        
        
        sig_ctext = await self.queue.get()
        ciphertext = await self.queue.get()
        self.peer_verify_key.verify(sig_ctext, ciphertext)

        
        sig_nounce = await self.queue.get()
        nounce = await self.queue.get()
        self.peer_verify_key.verify(sig_nounce, nounce)

        sig_tweak = await self.queue.get()
        tweak = await self.queue.get()
        self.peer_verify_key.verify(sig_tweak, tweak)
        
        print("\tCiphertext Received: "+str(ciphertext))

        if mode_TAE:
            plaintext = decrypt(ciphertext, tag, nounce, tweak, key, ad)
        else :
            aes = Cipher(algorithms.AES256(key), modes.CTR(nounce)).decryptor()
            plaintext = aes.update(ciphertext)
            xored = padding(tweak, plaintext)
            plaintext = aes.update(xored) + aes.finalize()
        
        
        print("Decrypted: "+str(plaintext)+"\n")

    async def print_agreed_key(self):
        print(self.agreed_key)


#### Função Main

Sobre esta função

Relativamente à utilização de uma única queue pelas duas instâncias da classe __channel__ foi feita de modo a simplificar a resolução deste exercicios. Funcionaria de modo análogo a utilização de 2 queues (assim como é num cenário real entre 2 utilizadores remotos), mas não era essencial para a resolução do exercício. As únicas mudanças seriam na inicialização das classes e leitura das queues.

In [386]:
async def main():
    
    queue = asyncio.Queue()

    
    emissor = channel(queue)
    receptor = channel(queue)

    
    await emissor.gen_keys()
    await receptor.gen_keys()

    
    await emissor.share_keys()
    await receptor.receive_keys()
    await receptor.share_keys()
    await emissor.receive_keys()

    
    
    

    if (emissor.agreed_key == receptor.agreed_key):
        print("Chave acordada: " + str(emissor.agreed_key) + "\n")
    else : 
        print(f"Chave não foi acordada\nChave emissor: {str(emissor.agreed_key)}\nChave recetor: {str(receptor.agreed_key)}")
        sys.exit("Chave não foi acordada")

    
    
    await emissor.send("Brave Sir Robin ran away. Bravely ran away away. . .")
    await receptor.receive()
    await receptor.send("A+ Recomendado!")
    await emissor.receive()

asyncio.run(main())




Chave acordada: b'\x00\x01\xaa%W"\xf0\xb8;02Q\xf3&\xf4\x1f\xb0_\x83\xed\x9d\xd5\xad\xc7\xbf\t\xc9\xbeo*SG'

Plaintext Sent: Brave Sir Robin ran away. Bravely ran away away. . .
	Ciphertext Sent: b'0\xbd\xc9]\xa3\xf2I\x16}[\xffaU\x85Y!&\x94?\x8e\x13\xba\x8f4$5\xbb\xb3CjC\xb0\x0b\xef\xdaJ\xa8\xf2{\x08n\x02\x8do@\x8dN/t\xdbq\x80r\xcd\xeeM\n\x15\xf9\xc1"\x1c&\xc8'
	Ciphertext Received: b'0\xbd\xc9]\xa3\xf2I\x16}[\xffaU\x85Y!&\x94?\x8e\x13\xba\x8f4$5\xbb\xb3CjC\xb0\x0b\xef\xdaJ\xa8\xf2{\x08n\x02\x8do@\x8dN/t\xdbq\x80r\xcd\xeeM\n\x15\xf9\xc1"\x1c&\xc8'
Decrypted: Brave Sir Robin ran away. Bravely ran away away. . .           

Plaintext Sent: A+ Recomendado!
	Ciphertext Sent: b'\x86~\x94/Z\xeb\x8ed\xe2\xdf\x8e]c{\x16\x0f\xde\x03<\x98\xe0\x1b\x91\xf9A}J\xb6t\xf9\n\x1b'
	Ciphertext Received: b'\x86~\x94/Z\xeb\x8ed\xe2\xdf\x8e]c{\x16\x0f\xde\x03<\x98\xe0\x1b\x91\xf9A}J\xb6t\xf9\n\x1b'
Decrypted: A+ Recomendado!                



## Testes

## CRIAR TESTES PARA ADICIONAR AQUI !!! !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!1
!!!!!!!!!!!!!!!!!!!!!!!!!!!!111
!!!!!!!!!!!!1
!!!!!!!!!!!!!!!!!111
!!!!!!!!!!!!!!!!!!!!

++ considerações finais

++ ESCREVER E COMENTAR CÓDIGO

++ utilizadar 2 queues????

++ ver o que foi comentadol 
ver se não me enganei e pus privada em vez de pública.

++ mudar nomes distos etc !!! 

ficha es

## __MUDAR TAMANHOS DOS BLOCOS tamanhos dos blocos, etc !!!__ MODO TAE + mudar cenas dar uma revisão geral, MUDAR FUNÇÕES ! etc

MUDAR NOMES E TUDO ISSO !!!! 

# NOTAS FINAIS : 

MUDAR ISTO PARA CYTHON

Multiprocessing

Ver trabalhos anos anteriores

Fazer testes

Explicar melhor os packages e kernel do miniconda neste arquivo

Explicar os 2 algoritmos:

It seems like you're trying to implement an AEAD (Authenticated Encryption with Associated Data) using Tweakable Block Ciphers with either AES-256 or ChaCha20 as the primitive block cipher. You also want to use this cipher to construct a private asynchronous information channel with key agreement done using X448 key exchange and Ed448 for agent authentication, including a key confirmation phase.

